In [1]:
from fos import util

In [2]:
snotel_gdf, _, _, _ = util.get_coords()
wrf_df = util.create_wrf_df(snotel_gdf)


ValueError: zero-size array to reduction operation maximum which has no identity